In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
# from mplfinance import candlestick_ohlc
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates



import numpy as np

from os.path import exists

from datetime import datetime
import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline
from decimal import *
getcontext().prec = 6
import seaborn as sns

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 300})


In [ ]:
def read_df_json(fname):
  # print(fname)
  df = pd.read_json(fname)
  df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

  df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
  df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
  return df

In [ ]:
symbol = 'ALGO_USDT'
data_folder = './freq-user-data/data/binance'

# timeframes = ['15m', '1h', '4h', '1d', '1w']
timeframes = ['1h', '1d', '1w']
dfs = [read_df_json(f'{data_folder}/{symbol}-{timeframe}.json') for timeframe in timeframes]
# dfs
# df_1h = read_df_json(f'{data_folder}/{symbol}-1h.json')
# df_15m = read_df_json(f'{data_folder}/{symbol}-15m.json')
# df_15m

In [17]:
timeframes[1:]

['1d', '1w']

In [ ]:
# pd.Timestamp('now').floor('D') + pd.Timedelta(-7, unit='D')

# start_time = datetime(2021,5,1)

start_time = pd.Timestamp('now') + pd.Timedelta(-100, unit='W')
# start_time
ddf = df.loc[df.index >= start_time].copy()
# ddf = df.loc[df['time'] >= start_time]
# ddf
dlen = len(ddf.index)
dlen

Timestamp('2021-03-01 00:00:00')

In [41]:

# sl_n2= IntSlider(description="n2", min=0, max=ceil(log(dlen, 2.0)+1), step=1, value=9)
# sl_w = IntSlider(description="w", min=0, max=ceil(dlen/pow(2,sl_n2.value-1)-1), step=1, value=1)
tf_keys = [f'tf_{tf}' for tf in timeframes]
bar_sliders = [IntSlider(description=timeframe, min=0, max=len(dfs[ind]), step=1, value=0) for ind, timeframe in enumerate(timeframes)]
ui = widgets.HBox(bar_sliders)


def on_barslider_change(change):
    # with output2:
    print(change)
slider_observers = [ bar_sliders[ind].observe(on_barslider_change, names='value') for ind, tf in enumerate(timeframes)]


current_date = pd.Timestamp(2021,3,1)
max_bars_plotted = 100
def printer(**kwargs):
    global current_time
    wdfs = [df.loc[df.date < current_date].tail(max_bars_plotted) for df in dfs]
    plt.close("all")
    
    with plt.rc_context():
        kwargs = dict(warn_too_much_data=1000)
        # fig = mpf.figure(figsize=(15,6))
        # ax1 = fig.add_subplot(2,1,1)
        # ax11 = ax1.twinx()

        # ap2 = [ 
        #         mpf.make_addplot(wdf, **{**kwargs, **dict(volume=False, style='binance',type='candle', figsize=(15, 6))}) 
        #         for wdf in wdfs[1:]     
        #     ]
        for wdf in wdfs:
            mpf.plot(wdf,**{**kwargs, **dict(volume=False, style='binance',type='candle', figsize=(15, 2))})
    plt.show()
    
out = widgets.interactive_output(printer, {
    tf_keys[ind]: bar_sliders[ind] for ind, timeframe in enumerate(timeframes)
})
display(ui, out)

Output()

In [ ]:
1 if len(wdf[~wdf.enter_price.isnull()]) else 0

In [ ]:
wdf.wick_btm_perc